## Building, training and testing a simple Linear regression model using pytorch

### installing dependencies

In [2]:
import torch
import torch.nn as nn

### Generating a mock dataset

In [3]:
weights = 0.7
bias = 0.3

X = torch.arange(0,20,0.2)
X.shape

torch.Size([100])

In [4]:
y = X * weights + bias
y.shape

torch.Size([100])

In [259]:
train_split = int(0.7 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([70]), torch.Size([30]), torch.Size([70]), torch.Size([30]))

### Building model using pytorch

In [260]:
class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1,requires_grad=True,dtype=torch.float))
        self.bias = nn.Parameter(torch.randn(1,requires_grad=True,dtype=torch.float))
    def forward(self,x:torch.Tensor)-> torch.Tensor:
        return self.weights * x + self.bias

In [261]:
torch.manual_seed(42)
model = LinearRegression()

In [262]:
model.state_dict()

OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])

In [263]:
list(model.parameters())

[Parameter containing:
 tensor([0.3367], requires_grad=True),
 Parameter containing:
 tensor([0.1288], requires_grad=True)]

#### setting loss function and optimizer

In [264]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.00001)

In [265]:
model.state_dict()

OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])

### Training the model

In [311]:
epochs = 10000
for i in range(epochs):
    model.train()
    y_pred = model(X_train)
    loss = loss_fn(y_pred,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.eval()
print(f"loss:{loss}")
model.state_dict()

loss:0.0004451313288882375


OrderedDict([('weights', tensor([0.7000])), ('bias', tensor([0.3000]))])

### Testing the model

In [314]:
with torch.inference_mode():
    y_pred_test = model(X_test)
    j_test = loss_fn(y_test,y_pred_test)
    print(f"test loss:{j_test}")

test loss:8.710225665709004e-05


In [315]:
y_pred_test,y_test

(tensor([10.0999, 10.2399, 10.3799, 10.5199, 10.6599, 10.7999, 10.9399, 11.0799,
         11.2199, 11.3599, 11.4999, 11.6399, 11.7799, 11.9199, 12.0599, 12.1999,
         12.3399, 12.4799, 12.6199, 12.7599, 12.8999, 13.0399, 13.1799, 13.3199,
         13.4599, 13.5999, 13.7399, 13.8799, 14.0199, 14.1599]),
 tensor([10.1000, 10.2400, 10.3800, 10.5200, 10.6600, 10.8000, 10.9400, 11.0800,
         11.2200, 11.3600, 11.5000, 11.6400, 11.7800, 11.9200, 12.0600, 12.2000,
         12.3400, 12.4800, 12.6200, 12.7600, 12.9000, 13.0400, 13.1800, 13.3200,
         13.4600, 13.6000, 13.7400, 13.8800, 14.0200, 14.1600]))

### Saving the model

In [318]:
from pathlib import Path
MODEL_DIR = Path("model")
MODEL_DIR.mkdir(parents=True,exist_ok=True)

MODEL_NAME = "pytorch_lr_model.pth"
MODEL_PATH = MODEL_DIR / MODEL_NAME

torch.save(obj=model.state_dict(),
           f=MODEL_PATH)

In [321]:
!ls model

pytorch_lr_model.pth


### loading saved model

In [322]:
l_model = LinearRegression()
l_model.state_dict()

OrderedDict([('weights', tensor([0.2345])), ('bias', tensor([0.2303]))])

In [323]:
l_model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [324]:
l_model.state_dict()

OrderedDict([('weights', tensor([0.7000])), ('bias', tensor([0.3000]))])

## Putting it all togather

In [325]:
import torch
from torch import nn

### 1.Data

In [378]:
X = torch.arange(0,20,0.2).unsqueeze(1)
y = 0.7*X+0.3
train_split = int(0.7 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([70, 1]),
 torch.Size([30, 1]),
 torch.Size([70, 1]),
 torch.Size([30, 1]))

### 2.Model

In [379]:
class LinearRegressionV2(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_layer = nn.Linear(in_features=1,
                                     out_features=1)
    def forward(self, x:torch.Tensor) -> torch.Tensor:
        return self.linear_layer(x)

torch.manual_seed(42)
model_v2 = LinearRegressionV2()
model_v2.state_dict()

OrderedDict([('linear_layer.weight', tensor([[0.7645]])),
             ('linear_layer.bias', tensor([0.8300]))])

In [380]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [381]:
model_v2.to(device)
next(model_v2.parameters()).device

device(type='cpu')

### 3.Training and testing

In [421]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model_v2.parameters(),
                            lr=0.001)

In [422]:
X_train,y_train,X_test,y_test = X_train.to(device), y_train.to(device), X_test.to(device), y_test.to(device)

In [430]:
torch.manual_seed(42)
epochs = 200
for epoch in range(epochs):
    model_v2.train()
    y_pred = model_v2(X_train)
    loss = loss_fn(y_train, y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model_v2.eval()
    with torch.inference_mode():
        test_pred = model_v2(X_test)
        test_loss = loss_fn(y_test, test_pred)

    if epoch % 10 == 0:
        print(f"epoch: {epoch}| train_loss: {loss}| test_loss: {test_loss}")

epoch: 0| train_loss: 0.0001306584890699014| test_loss: 0.11743389815092087
epoch: 10| train_loss: 0.0001306584890699014| test_loss: 0.11743389815092087
epoch: 20| train_loss: 0.0001306584890699014| test_loss: 0.11743389815092087
epoch: 30| train_loss: 0.0001306584890699014| test_loss: 0.11743389815092087
epoch: 40| train_loss: 0.0001306584890699014| test_loss: 0.11743389815092087
epoch: 50| train_loss: 0.0001306584890699014| test_loss: 0.11743389815092087
epoch: 60| train_loss: 0.0001306584890699014| test_loss: 0.11743389815092087
epoch: 70| train_loss: 0.0001306584890699014| test_loss: 0.11743389815092087
epoch: 80| train_loss: 0.0001306584890699014| test_loss: 0.11743389815092087
epoch: 90| train_loss: 0.0001306584890699014| test_loss: 0.11743389815092087
epoch: 100| train_loss: 0.0001306584890699014| test_loss: 0.11743389815092087
epoch: 110| train_loss: 0.0001306584890699014| test_loss: 0.11743389815092087
epoch: 120| train_loss: 0.0001306584890699014| test_loss: 0.117433898150920

In [426]:
model_v2.state_dict()

OrderedDict([('linear_layer.weight', tensor([[0.7000]])),
             ('linear_layer.bias', tensor([0.3001]))])

### 4.saving the model

In [431]:
from pathlib import Path
MODEL_DIR = Path("model")
MODEL_DIR.mkdir(parents=True,exist_ok=True)

MODEL_NAME = "pytorch_lr_model_2.pth"
MODEL_PATH = MODEL_DIR / MODEL_NAME

torch.save(obj=model.state_dict(),
           f=MODEL_PATH)